In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers

In [2]:
dataframe = pd.read_csv('ageC.csv')

In [3]:
#preprocess here
#Drop un-used columns.
dataframe['target'] = np.where(dataframe['class']=='>50K', 0, 1)
dataframe = dataframe.drop(columns=['fnlwgt', 'class', 'ageC', 'capital-loss', 
                                   'capital-lossB'])
dataframe.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,hours-per-week,native-country,target,capital-gainB
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,40,United-States,1,0
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,50,United-States,1,0
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,40,United-States,0,0
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,40,United-States,0,1
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,30,United-States,1,0


In [4]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [18]:
feature_columns = []

# numeric cols
for header in ['age', 'education-num', 'capital-gainB', 'hours-per-week']:
  feature_columns.append(feature_column.numeric_column(header))

# indicator_columns
indicator_column_names = ['workclass', 'education', 'marital-status', 'occupation',
                          'race', 'sex', 'native-country']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [6]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(dataframe, test_size=0.2)
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

print(len(train))

35166


In [28]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [29]:
checkpoint_path = "census/cp-{epoch:04d}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=0, 
    save_weights_only=True,
    save_freq=1000)


model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(32, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='RMSprop',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['acc',f1_m,precision_m, recall_m])


history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=[cp_callback], verbose=1)
#metrics=['acc',f1_m,precision_m, recall_m]

Epoch 1/50
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1097/1099 [============================>.] - ETA: 0s - loss: 0.3876 - acc: 0.8117 - f1_m: 0.8723 - precision_m: 0.8828 - recall_m: 0.8736WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'workclass': <tf.Tensor 'ExpandDims_12:0' shape=(None, 1) dtype=string>, 'education': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=string>, 'education-num': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=int64>, 'marital-status': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=string>, 'occupation': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'relationship': <tf.Tensor 'ExpandDims_10:0' shape=(None, 1) dtype=string>, 'race': <tf.Tensor 'ExpandDims_9:0' shape=(None, 1) dtype=string>, 'sex': <tf.Tensor 'ExpandDims_11:0' sh

1099/1099 [==============================] - 2s 2ms/step - loss: 0.3213 - acc: 0.8429 - f1_m: 0.8921 - precision_m: 0.9117 - recall_m: 0.8784 - val_loss: 0.3076 - val_acc: 0.8525 - val_f1_m: 0.9013 - val_precision_m: 0.9160 - val_recall_m: 0.8905
Epoch 20/50
1099/1099 [==============================] - 3s 2ms/step - loss: 0.3218 - acc: 0.8421 - f1_m: 0.8917 - precision_m: 0.9116 - recall_m: 0.8776 - val_loss: 0.3073 - val_acc: 0.8511 - val_f1_m: 0.8997 - val_precision_m: 0.9212 - val_recall_m: 0.8826
Epoch 21/50
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3216 - acc: 0.8415 - f1_m: 0.8913 - precision_m: 0.9112 - recall_m: 0.8772 - val_loss: 0.3059 - val_acc: 0.8557 - val_f1_m: 0.9040 - val_precision_m: 0.9143 - val_recall_m: 0.8972
Epoch 22/50
1099/1099 [==============================] - 3s 2ms/step - loss: 0.3209 - acc: 0.8425 - f1_m: 0.8919 - precision_m: 0.9115 - recall_m: 0.8780 - val_loss: 0.3075 - val_acc: 0.8470 - val_f1_m: 0.8957 - val_precision_m: 0.9285

In [30]:
#load testdata 
dataframe_test = pd.read_csv('adult_test.csv')

#prepare testdata
dataframe_test['target'] = np.where(dataframe_test['class']=='>50K', 0, 1)
dataframe_test['capital-gainB'] = np.where(dataframe_test['capital-gain']>5000, 1, 0)
dataframe_test = dataframe_test.drop(columns=['fnlwgt', 'class'])
test_ds = df_to_dataset(dataframe_test, shuffle=False, batch_size=batch_size)
print(len(test_ds))

153


In [31]:
loss, accuracy, f1_score, precision, recall = model.evaluate(test_ds)
print("Accuracy", accuracy, " Precision", precision, " Recall", recall)

Consider rewriting this model with the Functional API.
153/153 [==============================] - 0s 3ms/step - loss: 0.3110 - acc: 0.8536 - f1_m: 0.9019 - precision_m: 0.9055 - recall_m: 0.9014
Accuracy 0.8536036014556885  Precision 0.9054695963859558  Recall 0.9014052748680115
